In [1]:
!nvidia-smi

Fri Jun 10 21:26:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
%cd "/content/gdrive/MyDrive/ALPR-dipl"

/content/gdrive/MyDrive/ALPR-dipl


In [4]:
!unzip -q "detection.zip" -d "/content"

In [5]:
!unzip -q "recognition.zip" -d "/content"

In [6]:
!pip install -q pytorch-lightning torchvision

     |████████████████████████████████| 585 kB 4.8 MB/s 
     |████████████████████████████████| 419 kB 68.7 MB/s 
     |████████████████████████████████| 140 kB 61.3 MB/s 
     |████████████████████████████████| 596 kB 67.7 MB/s 
     |████████████████████████████████| 1.1 MB 58.4 MB/s 
     |████████████████████████████████| 144 kB 77.4 MB/s 
     |████████████████████████████████| 271 kB 72.3 MB/s 
     |████████████████████████████████| 94 kB 882 kB/s 


In [7]:
%cd "src"

/content/gdrive/MyDrive/ALPR-dipl/src


In [8]:
from importlib import reload
import os
from typing import Any, Dict

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import DataLoader, Dataset

import datasets
import models

In [ ]:
reload(datasets)
reload(models)

<module 'models' from '/content/gdrive/MyDrive/ALPR-dipl/src/models.py'>

In [9]:
pl.seed_everything(40)

Global seed set to 40


40

In [10]:
DATASETS = {
    "detection": datasets.DetectionDataset,
    "recognition": datasets.RecognitionDataset
}

COLLATE_FNS = {
    "recognition": datasets.RecognitionDataset.collate_fn
}

In [11]:
def create_dataloader(dataset_class: Dataset,
                      root: str,
                      train: bool,
                      batch_size: int,
                      collate_fn) -> DataLoader:
    dataset = dataset_class(root, train)
    dataloader = DataLoader(dataset,
                            batch_size=batch_size,
                            shuffle=True if train else False,
                            collate_fn=collate_fn)
    return dataloader

def create_trainer_and_fit(args: Dict[str, Any]) -> pl.Trainer:
    data_path = args["data_path"]
    dataset_class = DATASETS[args["dataset_name"]]

    collate_fn = COLLATE_FNS.get(args.get("collate_fn"))

    train_dl = create_dataloader(dataset_class,
                                 os.path.join(data_path, "train"),
                                 True,
                                 args["train_batch_size"],
                                 collate_fn)
    val_dl = create_dataloader(dataset_class,
                               os.path.join(data_path, "val"),
                               False,
                               args["val_batch_size"],
                               collate_fn)

    model = models.ALPRLightningModule(**args["alpr_module"])

    lr_cb = LearningRateMonitor(logging_interval="epoch")
    ckp_cb = ModelCheckpoint(filename="{epoch:02d}-{val_loss:.4f}",
                             monitor="val_loss",
                             save_weights_only=True)

    logger = TensorBoardLogger(**args["tb_logger"])

    trainer = pl.Trainer(**args["trainer"], 
                         callbacks=[lr_cb, ckp_cb],
                         logger=logger)
    
    trainer.fit(model, train_dl, val_dl)

    return trainer

In [12]:
RECOGNITION_CONFIG = {
    "data_path": "/content/recognition",
    "dataset_name": "recognition",
    "train_batch_size": 128,
    "val_batch_size": 128,
    "collate_fn": "recognition",
    "alpr_module": {
        "model_name": "recognizer",
        "model_hparams": {
            "n_classes": 37,
            "dropout": 0.3
        },
        "optimizer_name": "adamw",
        "optimizer_hparams": {
            "lr": 1e-3,
            "betas": (0.9, 0.999),
            "weight_decay": 1e-4,
        },
        "lr_scheduler_name": "reducelronplateau",
        "lr_scheduler_hparams": {
            "mode": "min",
            "factor": 0.1,
            "patience": 10,
            "min_lr": 1e-7
        },
        "loss_name": "ctc"
    },
    "tb_logger": {
        "save_dir": "../tb_logs",
        "name": "recognizer",
        "default_hp_metric": False
    },
    "trainer": {
        "gpus": 1,
        "max_epochs": 100
    }
}

In [13]:
create_trainer_and_fit(RECOGNITION_CONFIG)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ../tb_logs/recognizer
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | Recognizer | 551 K 
-------------------------------------
551 K     Trainable params
0         Non-trainable params
551 K     Total params
2.204     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/data.py:73: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 128. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [14]:
DETECTION_CONFIG = {
    "data_path": "/content/detection",
    "dataset_name": "detection",
    "train_batch_size": 128,
    "val_batch_size": 128,
    "alpr_module": {
        "model_name": "detector",
        "model_hparams": {
            "n_classes": 1,
            "dropout": 0.5
        },
        "optimizer_name": "adamw",
        "optimizer_hparams": {
            "lr": 1e-3,
            "betas": (0.9, 0.999),
            "weight_decay": 1e-4,
        },
        "lr_scheduler_name": "reducelronplateau",
        "lr_scheduler_hparams": {
            "mode": "min",
            "factor": 0.1,
            "patience": 8,
            "min_lr": 1e-7
        },
        "loss_name": "bce"
    },
    "tb_logger": {
        "save_dir": "../tb_logs",
        "name": "detector",
        "default_hp_metric": False
    },
    "trainer": {
        "gpus": 1,
        "max_epochs": 50
    }
}

In [15]:
create_trainer_and_fit(DETECTION_CONFIG)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ../tb_logs/detector
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type     | Params
-----------------------------------
0 | model | Detector | 3.8 M 
-----------------------------------
3.8 M     Trainable params
0         Non-trainable params
3.8 M     Total params
15.028    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]